In [14]:
import sys
sys.path.append("..")
from master import cds_utils as utils
# from master.cds_utils import *
import pandas as pd
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.preprocessing import MinMaxScaler
import os
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import roc_auc_score

In [8]:
## The metric for the model

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [23]:
class The_LSTM_Model:
    
    ## Preparing Dataset

    def preprocessing(self, dataframe_x,labels, size, features):
        list_of_x=[]
        list_of_y=np.zeros([1,1])
        
        for i in dataframe_x.values:
              list_of_x.append([i])
            
        list_of_y.fill(labels[2])

        # reshape input to be [samples, time steps, features]
        list_of_x = np.reshape(list_of_x, (1, size, features))

        return list_of_x,list_of_y

    ## Training and evaluating the model

    ## Train model in windows
    def train_model(self, X_train,y_train, size):
            # fix random seed for reproducibility
            np.random.seed(7)

            model.fit(X_train, y_train, epochs=3, batch_size=size)

 ## Evaluates model based on test set
    def evaluate_model(self, X_train,y_train,accuracy=0,iteration=0):

            # Final evaluation of the model
            scores = model.evaluate(X_train, y_train, verbose=0)

            #To get average accuracy of model
            accuracy+=scores[1]*100
            iteration+=1
            prediction=model.predict(X_train)

            return accuracy,iteration, prediction[0][0]


In [25]:
if __name__=='__main__':
    
    size=2
    spl_value=0.5
    
    lstm_model=The_LSTM_Model()
    # STEP 0: Check if combined dataset exists. If exists, GO TO STEP 4
    filename = '../master/AAPL - main.csv'
    if os.path.isfile(filename):
        dataset = pd.read_csv(filename, parse_dates=[0])
        dataset.set_index('Date', inplace=True)
    else:
        # STEP 1:: Aggregate financial data from discrete csv files
        #compile_financial('AAPL/')

        # STEP 2: Extract csv files into pandas dataframe
        # (1) financial dataframe
        financial = pd.read_csv('../master/AAPL - financial.csv', parse_dates=[1])
        financial = interpolate_data(financial, method='zero')
        # (2) price dataframe
        price = pd.read_csv('../master/AAPL - price.csv', parse_dates=[0]) # path to price 
        price.set_index('Date', inplace=True)
        # (3) technical dataframe
        technical = pd.read_csv('../master/AAPL - technical.csv', parse_dates=[0])
        technical.set_index('Date', inplace=True)
        # camel case column names for technical
        for old_column in technical:
            new_column = ' '.join([word.title() for word in old_column.split('_')])
            technical.rename(columns={old_column:new_column}, inplace=True)

        # STEP 3: Join different datasets based on overlapping dates
        dataset = combine_datasets(financial=financial, price=price, technical=technical)

    # STEP 4: Split the dataset into train and test
    train, test = utils.train_test_split(dataset, spl=spl_value)

    # STEP 5: Parse the train/test dataframe into a data generator
    data_gen = utils.data_generator(train, train_days=size, next='day')
    i=0

    lstm_out=200
    model = Sequential()
    model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
    model.add(LSTM(200, dropout= 0.5))
    # each 60 outputs will have a dimension of 100
    model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=[auc])
   
    scaler = MinMaxScaler(feature_range=(0, 1))
    y_is_one=0
    y_is_zero=0
    
    for x_train,y_train in data_gen:
        
        features=len(x_train.columns)
        
        if y_train[2]==1:
             y_is_one+=1
        else:
             y_is_zero+=1
            
        x,y=lstm_model.preprocessing(x_train, y_train,size, features)
        
        
        lstm_model.train_model(x,y,size)
    
    print(y_is_one,y_is_zero)
    
            
    data_gen = utils.data_generator(test, train_days=size, next='day')
    accuracy=0
    iteration=0
    y_true=[]
    y_predict=[]
    
    for x_test,y_test in data_gen:

        features=len(x_test.columns)
        y_true.append(y_test[2])
        x,y=lstm_model.preprocessing(x_test, y_test,size, features)
        accuracy,iteration, prediction=lstm_model.evaluate_model(x,y,accuracy,iteration)
        
        y_predict.append(prediction)
        
        
    print("Average")
    print(accuracy/iteration)
    
    the_score=roc_auc_score(y_true, y_predict)
    print(the_score)
    
#     summarize history for accuracy
#     plt.plot(history.history['acc'])
#     plt.plot(history.history['val_acc'])
#     plt.title('model accuracy')
#     plt.ylabel('accuracy')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
#     summarize history for loss
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
    



Epoch 1/3
1/1 [==============================] - 3s 3s/step - loss: 0.6948 - auc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6774 - auc: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6802 - auc: 0.0000e+00
Epoch 1/3
1/1 [==============================] - 0s 15ms/step - loss: 0.7168 - auc: 0.1667
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7085 - auc: 0.1667
Epoch 3/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7128 - auc: 0.1667
Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7123 - auc: 0.1667
Epoch 2/3
1/1 [==============================] - 0s 15ms/step - loss: 0.7121 - auc: 0.1667
Epoch 3/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7010 - auc: 0.2222
Epoch 1/3
1/1 [==============================] - 0s 15ms/step - loss: 0.7024 - auc: 0.2619
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.7007 - a

Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6254 - auc: 0.4211
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.5836 - auc: 0.4320
Epoch 3/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6703 - auc: 0.4338
Epoch 1/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6361 - auc: 0.4424
Epoch 2/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6031 - auc: 0.4522
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6267 - auc: 0.4606
Epoch 1/3
1/1 [==============================] - 0s 13ms/step - loss: 0.8258 - auc: 0.4493
Epoch 2/3
1/1 [==============================] - 0s 14ms/step - loss: 0.8425 - auc: 0.4384
Epoch 3/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7963 - auc: 0.4288
Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 0.5740 - auc: 0.4382
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6135 - auc: 0.4459

1/1 [==============================] - 0s 17ms/step - loss: 0.6931 - auc: 0.4411
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7136 - auc: 0.4428
Epoch 3/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6896 - auc: 0.4470
Epoch 1/3
1/1 [==============================] - 0s 15ms/step - loss: 0.6978 - auc: 0.4438
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6717 - auc: 0.4429
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6934 - auc: 0.4397
Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7156 - auc: 0.4412
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6971 - auc: 0.4454
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6890 - auc: 0.4494
Epoch 1/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6879 - auc: 0.4541
Epoch 2/3
1/1 [==============================] - 0s 17ms/step - loss: 0.6827 - auc: 0.4587
Epoch 3/3

Epoch 2/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7316 - auc: 0.4448
Epoch 3/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7176 - auc: 0.4451
Epoch 1/3
1/1 [==============================] - 0s 16ms/step - loss: 0.7069 - auc: 0.4463
Epoch 2/3
1/1 [==============================] - 0s 15ms/step - loss: 0.7543 - auc: 0.4448
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6897 - auc: 0.4469
Epoch 1/3
1/1 [==============================] - 0s 12ms/step - loss: 0.7348 - auc: 0.4459
Epoch 2/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7006 - auc: 0.4471
Epoch 3/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6906 - auc: 0.4491
Epoch 1/3
1/1 [==============================] - 0s 20ms/step - loss: 0.6884 - auc: 0.4511
Epoch 2/3
1/1 [==============================] - 0s 17ms/step - loss: 0.7323 - auc: 0.4502
Epoch 3/3
1/1 [==============================] - 0s 15ms/step - loss: 0.6974 - auc: 0.4521

1/1 [==============================] - 0s 13ms/step - loss: 0.7107 - auc: 0.4454
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6969 - auc: 0.4461
Epoch 1/3
1/1 [==============================] - 0s 11ms/step - loss: 0.6974 - auc: 0.4468
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6983 - auc: 0.4470
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6940 - auc: 0.4477
Epoch 1/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6773 - auc: 0.4493
Epoch 2/3
1/1 [==============================] - 0s 17ms/step - loss: 0.6908 - auc: 0.4499
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6456 - auc: 0.4527
Epoch 1/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7129 - auc: 0.4511
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7035 - auc: 0.4501
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.7270 - auc: 0.4481
Epoch 1/3

...

1/1 [==============================] - 0s 13ms/step - loss: 0.6732 - auc: 0.5055
Epoch 2/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6365 - auc: 0.5058
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6312 - auc: 0.5061
Epoch 1/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6619 - auc: 0.5064
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6545 - auc: 0.5067
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6146 - auc: 0.5070
Epoch 1/3
1/1 [==============================] - 0s 11ms/step - loss: 0.8008 - auc: 0.5068
Epoch 2/3
1/1 [==============================] - 0s 15ms/step - loss: 0.7854 - auc: 0.5065
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.7312 - auc: 0.5063
Epoch 1/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6020 - auc: 0.5066
Epoch 2/3
1/1 [==============================] - 0s 14ms/step - loss: 0.5967 - auc: 0.5069
Epoch 3/3

Epoch 2/3
1/1 [==============================] - 0s 16ms/step - loss: 0.7002 - auc: 0.5084
Epoch 3/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6875 - auc: 0.5082
Epoch 1/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6944 - auc: 0.5084
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6875 - auc: 0.5087
Epoch 3/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6888 - auc: 0.5089
Epoch 1/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6813 - auc: 0.5091
Epoch 2/3
1/1 [==============================] - 0s 15ms/step - loss: 0.6842 - auc: 0.5094
Epoch 3/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7101 - auc: 0.5096
Epoch 1/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7193 - auc: 0.5093
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7122 - auc: 0.5091
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6988 - auc: 0.5089

1/1 [==============================] - 0s 17ms/step - loss: 0.7176 - auc: 0.5082
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.7393 - auc: 0.5084
Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6700 - auc: 0.5082
Epoch 2/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6665 - auc: 0.5081
Epoch 3/3
1/1 [==============================] - 0s 20ms/step - loss: 0.6558 - auc: 0.5080
Epoch 1/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7135 - auc: 0.5082
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7276 - auc: 0.5083
Epoch 3/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7308 - auc: 0.5084
Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6657 - auc: 0.5083
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6624 - auc: 0.5082
Epoch 3/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6664 - auc: 0.5081
Epoch 1/3

Epoch 3/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7160 - auc: 0.5101
Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 0.7078 - auc: 0.5103
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.7074 - auc: 0.5105
Epoch 3/3
1/1 [==============================] - 0s 15ms/step - loss: 0.7136 - auc: 0.5107
Epoch 1/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6828 - auc: 0.5105
Epoch 2/3
1/1 [==============================] - 0s 15ms/step - loss: 0.6852 - auc: 0.5104
Epoch 3/3
1/1 [==============================] - 0s 15ms/step - loss: 0.6882 - auc: 0.5102
Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 0.6663 - auc: 0.5101
Epoch 2/3
1/1 [==============================] - 0s 15ms/step - loss: 0.6738 - auc: 0.5099
Epoch 3/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6566 - auc: 0.5098
Epoch 1/3
1/1 [==============================] - 0s 15ms/step - loss: 0.6736 - auc: 0.5097

Epoch 1/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6540 - auc: 0.5076
Epoch 2/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6525 - auc: 0.5075
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6552 - auc: 0.5074
Epoch 1/3
1/1 [==============================] - 0s 17ms/step - loss: 0.7202 - auc: 0.5076
Epoch 2/3
1/1 [==============================] - 0s 15ms/step - loss: 0.7356 - auc: 0.5077
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.7401 - auc: 0.5077
Epoch 1/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6575 - auc: 0.5076
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6490 - auc: 0.5075
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 0.6483 - auc: 0.5075
696 574
Average
50.676024762664255
0.5304752079018957


# AUC
# Daily
0.5304752079018957

In [ ]:
 # summarize history for accuracy
print(history.history['acc'])
# print(history.keys())
plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
    # summarize history for loss
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#0 451 0 792

# Daily - 2 Days
# 5 576 4 685

# Daily - 10 Days
# 0 578 0 684

# Daily- 2 Days
# 200 and then 300 dimension
# 45 536 69 620
# 52.3%

#
# Average
# 54.22257300710339

#Average, 200 then 300
##54.40944881889764
## 16 565 14 675

## 200 then 200
## 65 516 100 589
# Average
# 51.496062992125985

2 days
## 
Average
53.78486055776892
22 95 21 113
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200,dropout= 0.5, return_sequences=True))
model.add(LSTM(200))

Average
53.386454183266935
0 117 0 134
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(250,dropout= 0.5, return_sequences=True))
model.add(LSTM(250))

Average
53.386454183266935
0 117 0 134
4 layers

5 days
Average
52.82258064516129
0 117 0 131

30 days monthly
Average
55.70776255707762
0 97 0 122

# Findings

a/b ratio split where a refers to train set and b refers to test set

Accuracy of:

Average
52.84974093264249
On test data with 90/10 ratio split and 100 neurons in LSTM layer

Average
52.84974093264249
On test data with 90/10 ratio split and 200 neurons in LSTM layer

Average
53.960396039603964
On test data with 50/50 ratio split and 200 neurons in LSTM layer

Average
53.68349249658936
On test data with 40/60 ratio split and 200 neurons in LSTM layer

Average
53.68349249658936
On test data with 40/60 ratio split and 200 neurons in LSTM layer as well as addition of Conv1D and MaxPool

Average
53.6834924965893
On test data with 40/60 ratio split and 200 neurons in LSTM layer as well as addition of Conv1D and MaxPool, dropout_U = 0.2, dropout_W = 0.2

Average
45.78043315907394
On test data with 45/60 ratio split and 100 neurons in LSTM layer and dropout=0.5

Average
53.960396039603964

Average
54.106280193236714
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 30, 17))

Average
54.199683042789225
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 10, 17))

Average
54.22257300710339
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 5, 17))


Average
54.209284028324156
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 1, 17))

Average
54.488188976377955
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 2, 17))

Average
54.21591804570528
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 3, 17))

## Weekly
Average
57.03001579778831
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 2, 17))

Average
57.31319554848967
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 10, 17))


## Monthly
Average
63.716814159292035
On test data with 50/50 ratio split and 100 neurons in LSTM layer and dropout=0.5
model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
model.add(LSTM(200, dropout= 0.5))
list_of_x = np.reshape(list_of_x, (1, 10, 17))

In [ ]:
## Movie Example

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))